# 토큰화

예시 : 어제 카페 갔었어
- 단어 단위 -> 어제,카페,갔었어 / 어제,카페,갔었,어
- 문자 단위  -> 어,제,카,페,갔,었,어
- 서브워드 단위 : ex) Byte Pair Encoding(BPE), wordpiece

## BPE 어휘집합
: pre-tokenize 후에 초기 어휘 집합에서 토큰을 2개 씩 (bigram 쌍) 묶어 나열 -> 높은 빈도는 어휘집합에 추가 -> 미리 정한 어휘 집합의 크기를 달성한다면 BPE 어휘집합 구축 절차를 마침<br>
=> vocab.json으로 어휘집합을 저장, merge.txt로 병합이력을 모아둠

## BPE 토큰화
ex) 문장 : pug bug mug<br>
병합이력에 u g가 높은 우선 순위로 존재한다 가정 -> p,ug,b,ug,m,ug<br>
-> m이 어휘집합에 존재하지 않는다 가정 -> p,ug,b,ug,\<unk>,ug

=> 이런 경우 어휘 잡합의 크기를 합리적으로 유지하면서도 어휘를 구축할 때 보지 못했던 단어(신조어 등)에 대해 유의미한 분절을 수행할 수 있음

## wordpiece
: 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점에서 BPE와 유사하지만 문자열 병합의 기준이 '빈도'가 아닌 '우도'<br>
병합 후보가 a,b일 때 판단의 근거가 되는 값을 계산하는 방법 = ${\#ab/n}\over{(\#a/n)*(\#b/n)}$<br>
\#은 각 문자열의 빈도수, n은 전체 글자 수, 즉 분자는 ab가 연이어 등장할 확률, 분모는 a,b가 각각 등장할 확률의 곱

wordpiece는 토큰화도 BPE와 약간 다르다. BPE가 merges.txt와 vocal.json을 사용하는데 반해 wordpiece는 vocab.txt만 가지고 토큰화를 진행 <br>
-> 분석 대상 어절에 어휘 집합에 있는 서브워드가 포함돼 있을 경우에만 해당 서브워드를 어절에서 분리한다. 단 이러한 서브워드가 여럿 있을 경우 가장 긴 서브워드를 선택한다. <br>
-> 어절의 나머지 어휘 집합에 있는 서브워드를 다시 찾고(최장 일치 기준) 또 분리한다. <br>
(분석 대상 문자열에서 서브워드 후보가 하나도 없으면 해당 문자열 전체를 미등록 단어로 취급한다.)


In [ ]:
!pip install ratsnlp

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
# 네이버 영화 리뷰 NSMC
from Korpora import Korpora
nsmc = Korpora.load('nsmc', force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 29.0MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 12.4MB/s]                            


In [ ]:
import os
for path, lines in zip(['/content/train.txt', '/content/test.txt'],
                       [nsmc.train.get_all_texts(), nsmc.test.get_all_texts()]):
    with open(path, 'w',encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')


In [ ]:
!pip install tokenizers
!pip install transformers

# GPT 토크나이저(BPE) 구축
- 단 문자 단위가 아니라 유니코드 바이트 수준으로 어휘 집합을 구축하고 토큰화 진행 (미등록 토큰 문제에서 비교적 자유로움)
- 유니코드(UTF-8) : 1바이트를 10진수로 표현하면 0~255의 256개 정수가 되는데 이 정수 각각을 특정 문자로 매핑한 것임

어휘 집합 구축 대상 말뭉치를 바이트 단위로 변환하고 이들을 문자 취급해 가장 자주 등장한 문자열을 병합하는 방식으로 어휘 집합을 만드는 것임, 토큰화 역시 문자열을 바이트 단위로 변환한 뒤 수행한다.

In [ ]:
os.makedirs('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe', exist_ok=True)

In [ ]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(files=['/content/train.txt','/content/test.txt'],
                        vocab_size=10000,                   # 어휘 집합 크기 조절
                        special_tokens=['[PAD]'])           # 특수 토큰 추가
bytebpe_tokenizer.save_model('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe')

['/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe/vocab.json',
 '/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe/merges.txt']

# Bert 토크나이저
Bert는 워드피스 토크나이저를 사용

In [ ]:
os.makedirs('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/wordpiece', exist_ok=True)

In [ ]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase= False)
wordpiece_tokenizer.train(files=['/content/train.txt','/content/test.txt'],
                        vocab_size=10000)                   # 어휘 집합 크기 조절
wordpiece_tokenizer.save_model('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/wordpiece')

['/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe/vocab.txt']

# GPT 입력값 만들기

In [6]:
# GPT 입력값 만들기 : BPE 어휘 집합(vocab.json)과 바이그램 쌍의 병합 우선순위(merge.txt)

from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/bbpe')
tokenizer_gpt.pad_token = "[PAD]"

In [9]:
# 예시

sentences = ['아 더빙..진짜 짜증나네요 목소리',
             '흠...포스터보고 초딩영화줄...오버연기조차 가볍지 않구나',
             '별루 였다..']
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
print(sentences[0])
print(tokenized_sentences[0])

아 더빙..진짜 짜증나네요 목소리
['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'ì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬']


In [ ]:
# 실제 GPT 모델 입력은 다음과 같이 만든다
batch_inputs = tokenizer_gpt(
    sentences,
    padding='max_length',           # 문장 최대 길이에 맞춰 패딩
    max_length=12,                  # 문장의 토큰 기준 최대 길이
    truncation=True                 # 문장 잘림 허용 옵션
)


결과로는 input_ids, attention_mask가 만들어짐
- input_ids : 토큰화 결과를 가지고 각 토근을 인덱스로 바꾼 것임
- 어휘 집합(vocab.json)을 확인해 보면 각 어휘 순서대로 나열된 것을 확인할 수 있는데, 이 순서가 바로 인덱스임

In [19]:
# 토큰1~토큰12까지의 대한 인덱스를 출력
# max_length에 따라 12로 길이가 맞춰짐
# 이보다 짧은 문장1과 문장3은 뒤에 [PAD] 토큰에 해당하는 인덱스0이 붙었음 (일종의 dummy 토큰으로 길이를 맞춰주는 역할을 함)
# 문장 2는 원래 토큰 길이가 15였는데 truncate=True 옵션으로 잘림
print('원래 길이 : ',len(tokenized_sentences[0]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][0])
print('원래 길이 : ',len(tokenized_sentences[1]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][1])
print('원래 길이 : ',len(tokenized_sentences[2]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][2])

원래 길이 :  7 ,	토큰에 대한 인덱스 :  [334, 2338, 263, 663, 4055, 464, 3808, 0, 0, 0, 0, 0]
원래 길이 :  15 ,	토큰에 대한 인덱스 :  [3693, 336, 2876, 758, 2883, 356, 806, 336, 9875, 875, 2960, 7292]
원래 길이 :  4 ,	토큰에 대한 인덱스 :  [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]


- attention_mask는 일반 토큰이 자리한 곳 (1)과 패딩 토큰이 자리한 곳 (0)을 구분해 알려줌

In [21]:
print(batch_inputs['attention_mask'][0])
print(batch_inputs['attention_mask'][1])
print(batch_inputs['attention_mask'][2])

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


# Bert 입력값 만들기

In [22]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained('/gdrive/MyDrive/Colab Notebooks/Do it 자연어처리/wordpiece',
                                               do_lower_case=False)
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]
print(sentences[0])
print(tokenized_sentences[0])

아 더빙..진짜 짜증나네요 목소리
['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리']


In [ ]:
# 실제 Bert 모델 입력은 다음과 같이 만듦
batch_inputs = tokenizer_bert(
    sentences,
    padding='max_length',           # 문장 최대 길이에 맞춰 패딩
    max_length=12,                  # 문장의 토큰 기준 최대 길이
    truncation=True                 # 문장 잘림 허용 옵션
)

In [23]:
# 토큰의 2와 3인 인덱스는 <CLS>, <SEP>를 가리킴
print('원래 길이 : ',len(tokenized_sentences[0]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][0])
print('원래 길이 : ',len(tokenized_sentences[1]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][1])
print('원래 길이 : ',len(tokenized_sentences[2]),',\t토큰에 대한 인덱스 : ', batch_inputs['input_ids'][2])

원래 길이 :  8 ,	토큰에 대한 인덱스 :  [2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0]
원래 길이 :  19 ,	토큰에 대한 인덱스 :  [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1051, 16, 3]
원래 길이 :  4 ,	토큰에 대한 인덱스 :  [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]


In [24]:
print(batch_inputs['attention_mask'][0])
print(batch_inputs['attention_mask'][1])
print(batch_inputs['attention_mask'][2])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [27]:
# segment에 해당하는 것으로 모두 0임
# Bert 모델은 기본적으로 문서(혹은 문장) 2개를 입력받는데, 둘은 token_type_ids로 구분함
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]